In [16]:
from builtins import type

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

In [6]:
from keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    'dataset',
    image_size=(256, 256),
    batch_size=1,
    seed=123,
    validation_split=0.2,
    subset="training",
    label_mode='categorical'
)

val_dataset = image_dataset_from_directory(
    'dataset',
    image_size=(256, 256),
    batch_size=1,
    seed=123,
    validation_split=0.2,
    subset="validation",
    label_mode='categorical'
)

Found 6570 files belonging to 3 classes.
Using 5256 files for training.
Found 6570 files belonging to 3 classes.
Using 1314 files for validation.


In [7]:
for elem in train_dataset.take(1):
    print(elem)

(<tf.Tensor: shape=(1, 256, 256, 3), dtype=float32, numpy=
array([[[[231., 225., 229.],
         [234., 228., 232.],
         [236., 230., 234.],
         ...,
         [219., 212., 220.],
         [230., 223., 231.],
         [223., 216., 224.]],

        [[228., 222., 226.],
         [229., 223., 227.],
         [229., 223., 227.],
         ...,
         [229., 222., 230.],
         [226., 219., 227.],
         [224., 217., 225.]],

        [[240., 234., 238.],
         [238., 232., 236.],
         [235., 229., 233.],
         ...,
         [238., 231., 239.],
         [226., 219., 227.],
         [228., 221., 229.]],

        ...,

        [[204., 193., 199.],
         [200., 189., 195.],
         [201., 190., 196.],
         ...,
         [244., 234., 242.],
         [196., 186., 194.],
         [204., 194., 202.]],

        [[212., 201., 207.],
         [198., 187., 193.],
         [208., 197., 203.],
         ...,
         [232., 222., 230.],
         [228., 218., 226.],
        

In [12]:
from keras.applications import vgg16

conv_base = vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(256, 256, 3))

conv_base.trainable = False

58889256/58889256 [==============================] - 33s 1us/step


In [14]:
 conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [28]:
from keras import layers

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(256,256, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [29]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', mode='max', patience=10,
                           verbose=1)
checkpoint = ModelCheckpoint('/temp/tomato_multi_model-{epoch:02d}-{val_accuracy:.3f}.hdf5',
                             monitor='val_accuracy', mode='max',
                             verbose=1, save_best_only=True)
callbacks_list = [early_stop, checkpoint]

In [30]:
# callbacks = [
#     keras.callbacks.ModelCheckpoint(
#         filepath="feature_extraction_with_data_augmentation.keras",
#         save_best_only=True,
#         monitor="val_loss")
# ]

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=callbacks_list)

Epoch 1/50
 212/5256 [>.............................] - ETA: 2:01:17 - loss: 714.8477 - accuracy: 0.4717

KeyboardInterrupt: 